#### 计算两张图片之间的相似度

原理：利用VGG16网络抽像出来的特征做相似度比较。提一下，这里用到的特征是指block5_pool层所抽象的特征，也即是最后一个池化层。在keras提供的VGG16网络api中很容易做到这一点，只需要在初始化VGG16网络的时候设置include_top=False，就能丢弃掉最后的全连接层，最后的输出层便是最后一个池化层。

In [7]:
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image


import ssl
ssl._create_default_https_context = ssl._create_unverified_context # 下载模型的时候不想进行ssl证书校验

!export TF_CPP_MIN_LOG_LEVEL=2  # 屏蔽警告信息 （这是在jupyter中执行的shell命令，可以去掉这行不影响结果）

# 初始化VGG16模型 include_top=False 是指不保留顶层的3个全连接网络层
model = VGG16(weights='imagenet', include_top=False)

def get_feature(path):
    """
    VGG16模型抽取图片特征
    """
    # 图片转成PIL的Image对象，并且对图片做了缩放
    img = image.load_img(path, target_size=(224, 224))

    # 图片转成矩阵、并扩充了维度、最后是预处理
    predict_img = preprocess_input(np.expand_dims(image.img_to_array(img),0))

    # 丢入vgg16网络做特征抽取，最后返回特征并展平成一维向量方便计算余弦相似度
    return model.predict(predict_img).flatten()

def cos_sim(a, b):
    """
    计算两个向量之间的余弦相似度
    """
    a = np.mat(a)
    b = np.mat(b)
    return float(a * b.T) / (np.linalg.norm(a) * np.linalg.norm(b))

In [9]:
image_path = ['/Users/liuliangjun/Downloads/1.jpg', '/Users/liuliangjun/Downloads/2.jpg', '/Users/liuliangjun/Downloads/3.jpg']
ft1, ft2, ft3 = [get_feature(p) for p in image_path]

In [11]:
# 图1、图2相似度
cos_sim(ft1, ft2)

0.9247887575725511

In [12]:
# 图2、图3相似度
cos_sim(ft2, ft3)

0.9832494987242002

In [13]:
# 图1、图3相似度
cos_sim(ft1, ft3)

0.9130592505927942